In [6]:
import numpy as np
from sklearn.datasets import load_iris

In [2]:
# Construct a list that contains the number of neurons of each layer in your network.
structure = input("Enter the number of neurons of each layer in your network").split(' ')
structure = [int(x) for x in structure]
print(structure)    

[5, 4, 2]


In [3]:
# Construct a list that contains all the bias vectors of your network(_n means it has all vectors)
B_n = [np.random.randn(l, 1) for l in structure[1:]]
print("bias vector of each layer\n")
for B in B_n:
    print(B)
    print()

# Construct a list that contains all the weight matrices of your network(_n means it has all matrices)
W_n = [np.random.randn(l, next_l) for l, next_l in zip(structure[:-1], structure[1:])]
print("weight matrix of each layer\n")
for W in W_n:
    print(W)
    print()

bias vector of each layer

[[ 1.15797002]
 [ 0.95510743]
 [-0.77606375]
 [ 1.8599546 ]]

[[ 1.25400975]
 [-0.89762156]]

weight matrix of each layer

[[ 1.67411123  0.07273841 -0.98050266  0.70966027]
 [ 1.09129315  0.95320422  0.95963297  1.4114554 ]
 [-0.20763693 -0.79877846  1.17409124  0.40712783]
 [-0.14473857 -0.49760855 -1.25920288 -0.67147933]
 [ 0.22188607 -1.87498316  0.91089475 -0.291808  ]]

[[-0.66944414 -0.95429962]
 [ 0.46144331 -1.65526126]
 [ 2.31783589 -0.39629406]
 [-0.40425594  0.93366671]]



##### Load training and test datasets

In [12]:
load_iris().keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [16]:
data = load_iris()['data']
target = load_iris()['target']

print(type(data), type(target))
print(data.shape, target.shape)
print(len(data), len(target))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(150, 4) (150,)
150 150


In [15]:
# Randomly shuffle the data
np.random.seed(2023)

np.random.choice(data, size=len(data), replace=False)

ValueError: a must be 1-dimensional

##### Forward Pass

In [5]:
# Provide sigmoid and sigmoid_derivative function
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

In [53]:
x = np.array([2, 5])

In [75]:
np.array(B_n[0]).reshape(-1)

array([-0.79470329, -0.79819553,  0.2514908 ])

In [76]:
np.dot(W_n[0].T, x) + np.array(B_n[0]).reshape(-1)

array([3.27463785, 2.01854144, 0.66759429])

In [77]:
# Create two lists that contain pre and post activation vector of each layer, respectively
Z_n, A_n = [], []

# Forward pass layer by layer from L=0(First hidden layer) thru L=H(Output layer)
for i, (b, W) in enumerate(zip(B_n, W_n)):
    if i == 0:
        z = np.dot(np.array(W).T, x) + np.array(b).reshape(-1)
    else:
        z = np.dot(np.array(W).T, a) + np.array(b).reshape(-1)
    a = sigmoid(z)

    Z_n.append(z)
    A_n.append(a)

In [78]:
for z, a in zip(Z_n, A_n):
    print(np.round(z, 4))
    print(np.round(a, 4))
    print()

[3.2746 2.0185 0.6676]
[0.9635 0.8827 0.661 ]

[ 2.595  -0.0962]
[0.9305 0.476 ]



In [83]:
q = np.array([[0.54686406, 0.16518578, -0.87594857], 
              [ 0.5951226, 0.49727308,  0.43360012]])
r = np.array([-0.79470329, -0.79819553, 0.2514908])
y = np.dot(q.T, x) + r
u = sigmoid(y)

print(y)
print(u)

[3.27463783 2.01854143 0.66759426]
[0.96354842 0.88273011 0.66096426]
